# 🎨 ComfyUI + Domain Cá Nhân (Cloudflare Tunnel)

**Tính năng:**
- ✅ ComfyUI đầy đủ
- ✅ Cloudflare Tunnel (domain riêng)
- ✅ Keep Alive chống tự tắt
- ✅ Lưu models vào Google Drive

**Yêu cầu:** GPU T4/A100

In [ ]:
#@title 1️⃣ 🔄 KEEP ALIVE - Chống Colab tự tắt
import time, random, threading, subprocess, gc
from IPython.display import display, Javascript, HTML

display(Javascript('''
(function() {
    let count = 0;
    const activities = [
        () => document.querySelector("colab-connect-button")?.click(),
        () => window.scrollBy(0, Math.random()*50-25),
        () => document.dispatchEvent(new MouseEvent('mousemove', {
            clientX: Math.random()*window.innerWidth,
            clientY: Math.random()*window.innerHeight
        })),
        () => document.dispatchEvent(new KeyboardEvent('keydown', {key:'Shift'})),
        () => document.querySelector("#output-area")?.click()
    ];
    function run() {
        count++;
        for(let i=0; i<3; i++) activities[Math.floor(Math.random()*activities.length)]();
        var el = document.getElementById('ka-indicator');
        if(el) el.innerHTML = '🔄 '+count;
        setTimeout(run, 20000 + Math.random()*20000);
    }
    var div = document.createElement('div'); div.id='ka-indicator';
    div.innerHTML = '🔄 ON';
    div.style.cssText = 'position:fixed;top:10px;right:10px;background:linear-gradient(135deg,#667eea,#764ba2);color:white;padding:10px 20px;border-radius:25px;font-size:13px;font-weight:bold;z-index:9999;';
    document.body.appendChild(div);
    run();
})();
'''))

def py_activity():
    while True:
        time.sleep(random.randint(30,90))
        random.choice([gc.collect, lambda:len(dir()), lambda:time.time()])()
threading.Thread(target=py_activity, daemon=True).start()

def shell_activity():
    while True:
        time.sleep(random.randint(45,120))
        subprocess.run('echo alive', shell=True, capture_output=True)
threading.Thread(target=shell_activity, daemon=True).start()

print("✅ KEEP ALIVE ACTIVATED!")

In [ ]:
#@title 2️⃣ Kiểm tra GPU
!nvidia-smi
import torch
print(f"\n✅ CUDA: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

In [ ]:
#@title 3️⃣ Cài đặt ComfyUI
%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q -r requirements.txt
!pip install -q xformers
print("\n✅ Đã cài ComfyUI!")

In [ ]:
#@title 4️⃣ Cài ComfyUI Manager
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd /content/ComfyUI
print("✅ Đã cài ComfyUI Manager!")

In [ ]:
#@title 5️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/ComfyUI/models/checkpoints
!mkdir -p /content/drive/MyDrive/ComfyUI/models/loras
!mkdir -p /content/drive/MyDrive/ComfyUI/models/vae
!mkdir -p /content/drive/MyDrive/ComfyUI/outputs

!rm -rf /content/ComfyUI/models/checkpoints
!rm -rf /content/ComfyUI/models/loras
!rm -rf /content/ComfyUI/models/vae
!rm -rf /content/ComfyUI/output

!ln -s /content/drive/MyDrive/ComfyUI/models/checkpoints /content/ComfyUI/models/checkpoints
!ln -s /content/drive/MyDrive/ComfyUI/models/loras /content/ComfyUI/models/loras
!ln -s /content/drive/MyDrive/ComfyUI/models/vae /content/ComfyUI/models/vae
!ln -s /content/drive/MyDrive/ComfyUI/outputs /content/ComfyUI/output

print("✅ Đã liên kết Google Drive!")

In [ ]:
#@title 6️⃣ Download Models (Tùy chọn)
download_sdxl = True  #@param {type:"boolean"}
download_sd15 = False  #@param {type:"boolean"}

%cd /content/ComfyUI/models/checkpoints

if download_sdxl:
    print("📥 Downloading SDXL...")
    !wget -q --show-progress -O sd_xl_base_1.0.safetensors \
        "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors"

if download_sd15:
    print("📥 Downloading SD 1.5...")
    !wget -q --show-progress -O v1-5-pruned-emaonly.safetensors \
        "https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors"

print("\n✅ Done!")
!ls -lh

In [ ]:
#@title 7️⃣ Cài Cloudflare Tunnel
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
print("✅ Đã cài cloudflared!")

In [ ]:
#@title 8️⃣ 🚀 CHẠY COMFYUI + CLOUDFLARE TUNNEL
import subprocess
import threading
import time

# ============================================
# 🔧 CẦN ĐIỀN THÔNG TIN BÊN DƯỚI
# ============================================
CLOUDFLARE_TUNNEL_TOKEN = ""  #@param {type:"string"}
YOUR_DOMAIN = ""  #@param {type:"string"}
# ============================================

if not CLOUDFLARE_TUNNEL_TOKEN or not YOUR_DOMAIN:
    print("❌ Chưa điền thông tin Cloudflare!")
    print("\n📋 Hướng dẫn:")
    print("1. Vào https://one.dash.cloudflare.com/")
    print("2. Zero Trust → Networks → Tunnels")
    print("3. Create tunnel → Copy token")
    print("4. Thêm Public Hostname với localhost:8188")
    print("5. Điền token và domain vào ô trên")
else:
    def run_cloudflared():
        subprocess.run(["cloudflared", "tunnel", "run", "--token", CLOUDFLARE_TUNNEL_TOKEN])

    tunnel_thread = threading.Thread(target=run_cloudflared, daemon=True)
    tunnel_thread.start()
    time.sleep(5)

    print("="*60)
    print(f"🌐 COMFYUI URL: https://{YOUR_DOMAIN}")
    print("="*60)

    %cd /content/ComfyUI
    !python main.py --listen 0.0.0.0 --port 8188

In [ ]:
#@title [Optional] Cài Custom Nodes
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/Fannovel16/comfyui_controlnet_aux.git
!git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus.git
!git clone https://github.com/ltdrdata/ComfyUI-Impact-Pack.git
!pip install -q -r comfyui_controlnet_aux/requirements.txt
print("\n✅ Đã cài custom nodes!")